In [ ]:
import requests
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

API_KEY = 'a975812056cf3fa64f80383505b4f94f'
BASE_URL = 'https://api.themoviedb.org/3'

def get_movie_id(movie_name):
    search_url = f"{BASE_URL}/search/movie?api_key={API_KEY}&query={movie_name}"
    response = requests.get(search_url)
    data = response.json()
    if data['results']:
        return data['results'][0]['id'], data['results'][0]['title']
    else:
        return None, None

def get_similar_movies(movie_id):
    similar_url = f"{BASE_URL}/movie/{movie_id}/similar?api_key={API_KEY}"
    response = requests.get(similar_url)
    data = response.json()
    if data['results']:
        movies = [{
            'id': movie['id'],
            'title': movie['title'],
            'vote_average': movie['vote_average'],
            'release_date': movie['release_date']
        } for movie in data['results']]
        return movies
    else:
        return []

def get_movie_details(movie_id):
    details_url = f"{BASE_URL}/movie/{movie_id}?api_key={API_KEY}"
    response = requests.get(details_url)
    return response.json()

def get_movie_cast(movie_id):
    credits_url = f"{BASE_URL}/movie/{movie_id}/credits?api_key={API_KEY}"
    response = requests.get(credits_url)
    data = response.json()
    if 'cast' in data:
        cast = [member['name'] for member in data['cast'][:5]]  
        return ', '.join(cast)
    else:
        return "No cast information available"

def visualize_similar_movies(movies, original_movie):
    df = pd.DataFrame(movies)
    df = df.sort_values(by='vote_average', ascending=False)
    
    titles = df['title'].tolist()
    ratings = df['vote_average'].tolist()

    sns.set(style="whitegrid")
    plt.figure(figsize=(10, 6))
    ax = sns.barplot(x=ratings, y=titles)
    ax.set_title(f"Movies Similar to '{original_movie}'", fontsize=16)
    ax.set_xlabel('Average Rating', fontsize=14)
    ax.set_ylabel('Movie Title', fontsize=14)
    
    plt.tight_layout()
    plt.show()

def main():
    movie_name = input("Please enter the movie you like: ") 
    movie_id, original_movie = get_movie_id(movie_name)
    
    if movie_id:
        similar_movies = get_similar_movies(movie_id)
        if similar_movies:
            movie_info = {
                'Property': ['Rating', 'Release Date', 'Overview', 'Cast']
            }
            
            for i, movie in enumerate(similar_movies):
                details = get_movie_details(movie['id'])
                overview = details.get('overview', 'No overview available').replace('\n', ' ')
                cast = get_movie_cast(movie['id']).replace('\n', ' ')
                movie_info[movie['title']] = [
                    movie['vote_average'],
                    movie['release_date'],
                    overview,
                    cast
                ]
            
            movie_df = pd.DataFrame(movie_info)
            movie_df.set_index('Property', inplace=True)
            pd.set_option('display.max_colwidth', 130) 
            print("\nMovie description")
            print(movie_df)
            visualize_similar_movies(similar_movies, original_movie) 
        else:
            print("Sorry, no similar movies found")
    else:
        print("Sorry, this movie could not be found, please try another movie you like!")

if __name__ == "__main__":
    main()